Submission probabilities tweaks

In [3]:
import numpy as np
import pandas as pd

In [43]:
df = pd.read_csv("../results/submission_merge__mean_cv=6_squeezenet_fold=0_mixed_cnn_2017-06-10-11-50.csv")

In [44]:
df.head()

,image_name,Type_1,Type_2,Type_3
0,0.jpg,0.222587,0.533222,0.244191
1,1.jpg,0.032502,0.686077,0.281421
2,10.jpg,0.022474,0.975495,0.002031
3,100.jpg,0.175312,0.770244,0.054444
4,101.jpg,0.523968,0.418640,0.057392


In [45]:
def foo(probas):
    imax = np.argmax(probas.values)
    indices = range(3)
    indices.remove(imax)
    probas[imax] = 0.95
#     probas[indices] = 0.025
    return probas

In [46]:
cols = ['Type_%i' % i for i in range(1, 4)]
df[cols] = df.loc[:, cols].apply(foo, axis=1)

In [47]:
import os
from datetime import datetime

info = 'merge__mean_cv=6_squeezenet_fold=0_mixed_cnn'

now = datetime.now()
sub_file = 'submission_' + info + '_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '_postproc.csv'
sub_file = os.path.join('..', 'results', sub_file)
df.to_csv(sub_file, index=False)

In [48]:
df.head()

,image_name,Type_1,Type_2,Type_3
0,0.jpg,0.222587,0.95000,0.244191
1,1.jpg,0.032502,0.95000,0.281421
2,10.jpg,0.022474,0.95000,0.002031
3,100.jpg,0.175312,0.95000,0.054444
4,101.jpg,0.950000,0.41864,0.057392


Merge two submissions

In [1]:
import numpy as np
import pandas as pd

In [2]:
df1 = pd.read_csv("../results/submission_fold=0_mixed_cnn_2017-06-08-21-49.csv")
df2 = pd.read_csv("../results/submission_mean_cv=6_squeezenet_2017-05-25-23-30.csv")

In [8]:
df1.head()

,image_name,Type_1,Type_2,Type_3
0,0.jpg,0.359289,0.511135,0.129576
1,1.jpg,0.032502,0.686077,0.281421
2,10.jpg,0.022474,0.975495,0.002031
3,100.jpg,0.175312,0.770244,0.054444
4,101.jpg,0.523968,0.418640,0.057392


In [9]:
df2.head()

,image_name,Type_1,Type_2,Type_3
0,0.jpg,0.222587,0.533222,0.244191
1,1.jpg,0.215771,0.542684,0.241544
2,10.jpg,0.330304,0.534667,0.135029
3,100.jpg,0.300007,0.633719,0.066274
4,101.jpg,0.475317,0.412831,0.111853


In [38]:
def merge(df1, df2):
    types = ['Type_1', 'Type_2', 'Type_3']
    ll = min(df1.shape[0], df2.shape[0])
    df_out = pd.DataFrame(columns=df1.columns)
    for i in range(ll):    
        proba1 = df1.loc[i, types]
        proba2 = df2.loc[i, types]        
        proba = proba1 if np.max(proba1) > np.max(proba2) else proba2    
        df_out.loc[i, :] = (df1.loc[i, 'image_name'], ) + tuple(proba)  
    return df_out

In [40]:
df3 = merge(df1, df2)

In [41]:
df3.head()

,image_name,Type_1,Type_2,Type_3
0,0.jpg,0.222587,0.533222,0.244191
1,1.jpg,0.0325019,0.686077,0.281421
2,10.jpg,0.0224738,0.975495,0.00203107
3,100.jpg,0.175312,0.770244,0.0544437
4,101.jpg,0.523968,0.41864,0.0573924


In [42]:
import os
from datetime import datetime

info = 'merge__mean_cv=6_squeezenet_fold=0_mixed_cnn'

now = datetime.now()
sub_file = 'submission_' + info + '_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
sub_file = os.path.join('..', 'results', sub_file)
df3.to_csv(sub_file, index=False)

Play with logloss function

In [2]:
import os
import sys
# Project
project_common_path = os.path.dirname('.')
project_common_path = os.path.abspath(os.path.join(project_common_path, '..', 'common'))
if not project_common_path in sys.path:
    sys.path.append(project_common_path)

In [4]:
import numpy as np
from metrics import logloss_mc

In [17]:
n=100

In [18]:
y_true = np.zeros((n, 3), dtype=np.uint8)
y_true[:, 0] = 1

In [58]:
y_pred = 0.33 * np.ones((n, 3))
index = np.random.randint(0, 3, size=n)

In [59]:
logloss_mc(y_true, y_pred)

1.0986122886681107

In [95]:
y_pred = 0.2 * np.ones((n, 3))
index = np.random.randint(0, 3, size=n)
y_pred[np.arange(n), index] = 0.85

In [96]:
logloss_mc(y_true, y_pred)

1.3406290095499604